In [1]:
import os
import re
import pandas as pd
import pickle
import collections
from collections import defaultdict
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen

from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

c:\programdata\miniconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
brd_name_lst = ['플렌느',
 '플랙커스',
 '킨더퍼페츠',
 '큐비스킨랩',
 '코코도르',
 '코디',
 '위가드',
 '오스트레일리안 네이티브 보타니컬스',
 '아베크힐',
 '비오쿠라',
 '볼루스파',
 '바비쉬',
 '바바파파베이비',
 '미넴보떼',
 '르나르도',
 '라꾸베',
 '데코뷰',
 '니드인',
 '나스브']

In [3]:
ssg_brd_df = pd.read_csv("ssg_brd_df.csv")

In [4]:
brd_id_lst = []
for brd_name in brd_name_lst:
    brd_id_lst.append(int(ssg_brd_df.loc[ssg_brd_df.brd_name == brd_name].brd_id))

In [5]:
brd_id_lst

[3000032430,
 3000021581,
 3000023468,
 3000043089,
 3000034072,
 2000021996,
 3000053129,
 3000037202,
 3000056375,
 1030770232,
 2011011573,
 1010000417,
 3000026369,
 3000045236,
 3000027667,
 2000011655,
 2000011062,
 3000049866,
 3000036863]

In [6]:
exception = []
brd_w_prd = {}

In [9]:
for brd_id in tqdm(brd_id_lst):

    prd_id_lst = []
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    socket.setdefaulttimeout(30)

    try:
        wd.get('http://www.ssg.com/disp/brandShop.ssg?brandId={}'.format(brd_id))
        time.sleep(3)

        html = wd.page_source
        soup = BeautifulSoup(html, "html.parser")

        try:
            last_page_num = int(re.findall('changePage\((\d*?)\)', str(soup.find("a", {"class": "btn_last on"})))[0])
        except:
            last_page_num = 1

        for page in range(1, last_page_num+1):
            wd.get('http://www.ssg.com/disp/brandShop.ssg?brandId={}&page={}'.format(brd_id, page))
            time.sleep(3)
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            
            cunit_t290 = soup.find_all("li", {"class": "cunit_t290"})
            
            if cunit_t290 != []:
                for i in cunit_t290:
                    prd_clickable = i.find_all("div", {"class": "title"})
                    prd_id_lst.append(int(re.findall('data-info="(\d*?)"', str(prd_clickable))[0]))

        brd_w_prd[brd_id] = prd_id_lst
        
    except Exception as e:
        exception.append(brd_id)
        print("에러가 발생했습니다.")
        print(brd_id, ": ", e)
        pass

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "c:\programdata\miniconda3\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "c:\programdata\miniconda3\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


Mozilla/5.0 (X11; CrOS i686 3912.101.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
There is no [win32] chromedriver for browser 95.0.4638 in cache
Get LATEST driver version for 95.0.4638
Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.17/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17]


Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2226.0 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


...



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2117.157 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.137 Safari/4E423F




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


In [12]:
def scrap_product_comments(soup):
    try:
        #comments
        comment_list_area = BeautifulSoup(str(soup.find_all("div", {"id": "comment_list_area"})), "html.parser")
        comment_total_cnt = int(comment_list_area.find("em", {"id": "gCommentTotalCnt"}).text.replace(',', ''))
        

        comment_table_area = BeautifulSoup(str(soup.find_all("table", {"summary": "전체 상품평 보여주는 표"})), "html.parser")

        ratings = BeautifulSoup(str(comment_table_area.find_all("span", {"class": "sp_cdtl cdtl_cmt_star"})), "html.parser")
        rating = [x.text for x in ratings.find_all("em")]

        users = BeautifulSoup(str(comment_table_area.find_all("td", {"class": "user"})), "html.parser")
        user = [x.text.strip() for x in users.find_all("div", {"class":"in"})]

        dates = BeautifulSoup(str(comment_table_area.find_all("td", {"class": "date"})), "html.parser")
        date = [x.text.strip() for x in dates.find_all("div", {"class":"in"})]

        comment_row = BeautifulSoup(str(comment_table_area.find_all("td", {"class": "desc_txt"})), "html.parser")
        review_typ_month = [x.find_all("span", {"class": "review_typ typ_month"}) for x in comment_row.find_all("td", {"class": "desc_txt"})]
        for i in range(len(review_typ_month)):
            if review_typ_month[i] != []:
                review_typ_month[i] = '한달사용리뷰'
            else:
                review_typ_month[i] = None

        cdtl_cmt_tx2 = BeautifulSoup(str(comment_row.find_all("div", {"class": "cdtl_cmt_tx2"})), "html.parser")
        purchase_option = [x.find_all("div", {"class": "cdtl_cmt_tx2"}) for x in comment_row.find_all("td", {"class": "desc_txt"})]

        for i in range(len(purchase_option)):
            if purchase_option[i] != []:
                purchase_option[i] = cdtl_cmt_tx2.find_all("div")[i].text
            else:
                purchase_option[i] = None

        comment_view = BeautifulSoup(str(comment_table_area.find_all("tr", {"class": "cdtl_cmt_view"})), "html.parser")

        review_eval = BeautifulSoup(str(comment_view.find_all("ul", {"class":"review_eval"})), "html.parser")
        evaluation_item = []
        evaluation = []

        for i in range(len(review_eval.find_all("ul"))):
            evaluation_item.append([x.text for x in review_eval.find_all("ul")[i].find_all("h5")]) #고객평가항목
            evaluation.append([x.text for x in review_eval.find_all("ul")[i].find_all("span")]) #고객평가항목평가

        comment_desc = [x.text.replace('\n', '') for x in comment_view.find_all("div", {"class": "desc"})]

        return rating, user, date, review_typ_month, evaluation_item, evaluation, comment_desc, purchase_option
    
    except:
        rating = None
        user = None
        date = None
        review_typ_month = None
        evaluation_item = None
        evaluation = None
        comment_desc = None
        purchase_option = None
        
        return rating, user, date, review_typ_month, evaluation_item, evaluation, comment_desc, purchase_option

def current_comment_navi_area(soup):
    return soup.find("div", {"id": "comment_navi_area"})

In [13]:
def scrap_ssg_product_info(brand_id, product_id):
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    socket.setdefaulttimeout(30)
    
    while True:
        
        try:
            wd.get('http://www.ssg.com/item/itemView.ssg?itemId={}'.format(product_id))
            wd.maximize_window()
            time.sleep(2)

            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")


            #product name
            prd_name = soup.find("h2", {"class": "cdtl_info_tit"}).text
            prd_price = soup.find("em", {"class": "ssg_price"}).text

            try:
                model_num = soup.find("p", {"class": "cdtl_model_num"}).text.split(': ')[-1]
            except:
                model_num = None

            #상품 필수 정보
            cdtl_cont_info = BeautifulSoup(str(soup.find_all("table", {"summary": "상품 필수정보 보여주는 표"})), "html.parser") 
            product__essential_info_items = [x.text for x in cdtl_cont_info.find_all("th", {"scope": "row"})]
            product__essential_info = [x.text for x in cdtl_cont_info.find_all("td")]

        #     prd_size = cdtl_cont_info.find_all("td")[0].text
        #     prd_specification = cdtl_cont_info.find_all("td")[1].text
        #     prd_date_of_use = cdtl_cont_info.find_all("td")[2].text
        #     prd_how_to_use = cdtl_cont_info.find_all("td")[3].text
        #     prd_manufacturer = cdtl_cont_info.find_all("td")[4].text
        #     prd_ingredients = cdtl_cont_info.find_all("td")[5].text
        #     prd_examination = cdtl_cont_info.find_all("td")[6].text
        #     prd_caution = cdtl_cont_info.find_all("td")[7].text
        #     prd_quality_assurance_standards = cdtl_cont_info.find_all("td")[8].text
        #     prd_consumer_consulting_number = cdtl_cont_info.find_all("td")[9].text
        #     prd_manufacture_country = cdtl_cont_info.find_all("td")[10].text

            #상품 옵션
            try:
                opt = BeautifulSoup(str(soup.find_all("div", {"id": "_ordOpt_area"})), "html.parser")
                opt_lst = BeautifulSoup(str(opt.find_all("ul", {"class", "cdtl_select_lst _drop_list"})), "html.parser")
                options = [x.text for x in opt_lst.find_all("span", {"class": "txt"})]  
            except:
                options = None

            #category
            try:
                location = BeautifulSoup(str(soup.find_all("div", {"id": "location"})), "html.parser")
                category = [x.text for x in location.find_all("a", {"class": "lo_arr"})]
            except:
                category = None

            comment_navi_area = current_comment_navi_area(soup)

            try:
                if comment_navi_area.find("a", {"class": "btn_last"}):
                    last_comment_page_num = int(re.findall("fn_GoCommentPage\('(\d*?)\\'\).*?", str(comment_navi_area.find("a", {"class": "btn_last"})))[0])
                else:
                    last_comment_page_num = int(comment_navi_area.find_all("a")[-1].text)
            except:
                last_comment_page_num = 1

            #first comment page scraping
            rating, user, review_date, review_typ_month, evaluation_item, evaluation, comment_desc, user_purchase_option = scrap_product_comments(soup)
            #print("first page: ", number)


            for i in range(last_comment_page_num-1):

                prev_btn_cnt = len(re.findall('onclick="fn_GoCommentPage', str(comment_navi_area).split('<strong title="현재위치"')[0]))
                wd.find_element_by_xpath("/html/body/div[4]/div[6]/div/div[2]/div[6]/div[1]/div/div[2]/div[2]/div[2]/div[4]/div[3]/a[{}]".format(prev_btn_cnt+1)).click()
                time.sleep(5)

                html = wd.page_source
                time.sleep(3)
                soup = BeautifulSoup(html, "html.parser")
                comment_navi_area = BeautifulSoup(str(soup.find("div", {"id": "comment_navi_area"})), "html.parser")

                rating_add, user_add, review_date_add, review_typ_month_add, evaluation_item_add, evaluation_add, comment_desc_add, user_purchase_option_add = scrap_product_comments(soup)

                rating = rating + rating_add
                user = user + user_add
                review_date = review_date + review_date_add
                review_typ_month = review_typ_month + review_typ_month_add
                evaluation_item = evaluation_item + evaluation_item_add
                evaluation = evaluation + evaluation_add
                comment_desc = comment_desc + comment_desc_add  
                user_purchase_option = user_purchase_option + user_purchase_option_add

            wd.close

            prd_info = {
                "brand_id": brand_id,
                "brand_name": None,
                "product_id": product_id,
                "product_name": prd_name,
                "product_price": prd_price,
                "model_number": model_num,
                "product__essential_info_items": product__essential_info_items,
                "product__essential_info": product__essential_info,
                "option": options,
                "category": category,
                'rating': rating,
                'user': user,
                'review_date': review_date,
                'review_typ_month': review_typ_month,
                'evaluation_item': evaluation_item,
                'evaluation': evaluation,
                'comment_desc': comment_desc,
                'user_purchase_option': user_purchase_option
            }

            return prd_info

        except urllib3.exceptions.ReadTimeoutError:
            time.sleep(5)

        except:
            prd_info = {
                "brand_id": brand_id,
                "brand_name": None,
                "product_id": product_id,
                "product_name": None,
                "product_price": None,
                "model_number": None,
                "product__essential_info_items": None,
                "product__essential_info": None,
                "option": None,
                "category": category,
                'rating': None,
                'user': None,
                'review_date': None,
                'review_typ_month': None,
                'evaluation_item': None,
                'evaluation': None,
                'comment_desc': None,
                'user_purchase_option': None
            }        
            print("에러가 발생했습니다.")
            return prd_info

In [14]:
ssg_prd_info = pd.DataFrame(columns = ['brand_id',
                                      'brand_name', 
                                      'product_id',
                                      'product_name',
                                      'product_price',
                                      'model_number',
                                        'product__essential_info_items',
                                        'product__essential_info',
                                       'option',
                                       'category',
                                      'rating',
                                      'user',
                                      'review_date',
                                      'review_typ_month',
                                      'evaluation_item',
                                      'evaluation',
                                      'comment_desc',
                                      'user_purchase_option'])

In [ ]:
idx = 0
for brd_id in tqdm(brd_w_prd.keys()):
    for prd_id in tqdm(brd_w_prd[brd_id]):
        prd_info = scrap_ssg_product_info(brd_id, prd_id)
        ssg_prd_info.loc[idx] = prd_info
        idx += 1

Mozilla/5.0 (X11; CrOS i686 3912.101.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.2 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Mozilla/5.0 (Windows NT 6.1; WOW64; rv:21.0) Gecko/20100101 Firefox/21.0




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache
